In [3]:
import random
import copy
import multiprocessing
import pandas as pd
import numpy as np
from deap import base
from deap import creator
from deap import tools
from datetime import datetime, timedelta

In [4]:
# Load files
path = '../Data/'
festival = 'IFEMA_Festival_Videojuegos_1/'
preferences = 'Preferencias/'
file_name= 'IFEMA_Festival_Videojuegos_v_5.csv'
ORIGINAL_EVENTO = pd.read_csv(path+festival+file_name)
file_name='PREFERENCIAS_US1.csv'
PREFERENCIAS = pd.read_csv(path+festival+preferences+file_name)
PREFERENCIAS = PREFERENCIAS.sort_values('Id_Recurso')
PREFERENCIAS.head(15)

,Id_Recurso,Preferencia
101,0,0.233295
88,1,0.358799
111,2,0.120212
102,3,0.227418
113,4,0.102221
107,5,0.184035
27,6,0.494820
114,7,0.044136
50,8,0.425795
89,9,0.358799


In [5]:
# Transform to date and hour formats
ORIGINAL_EVENTO['Hora_apertura_1'] = pd.to_datetime(ORIGINAL_EVENTO['Hora_apertura_1'], format='%H:%M')
ORIGINAL_EVENTO['Hora_cierre_1'] = pd.to_datetime(ORIGINAL_EVENTO['Hora_cierre_1'], format='%H:%M')
ORIGINAL_EVENTO['Fecha_1'] = pd.to_datetime(ORIGINAL_EVENTO['Fecha_1'], format='%d/%m/%Y')

ORIGINAL_EVENTO['Hora_apertura_2'] = pd.to_datetime(ORIGINAL_EVENTO['Hora_apertura_2'], format='%H:%M')
ORIGINAL_EVENTO['Hora_cierre_2'] = pd.to_datetime(ORIGINAL_EVENTO['Hora_cierre_2'], format='%H:%M')
ORIGINAL_EVENTO['Fecha_2'] = pd.to_datetime(ORIGINAL_EVENTO['Fecha_2'], format='%d/%m/%Y')

In [6]:
# Get number of days of the event
DAYS_EVENT = (max(ORIGINAL_EVENTO.Fecha_2) - min(ORIGINAL_EVENTO.Fecha_1)).days + 1

In [7]:
# Exclude resources of more than 6 hours of duration
drop_list = []
for i in range(0, max(ORIGINAL_EVENTO['Código'])):
    
    # Get open time for the first resource
    ha1 = ORIGINAL_EVENTO.Hora_apertura_1[i]
                        
    # Get close time for the first resource
    hc1 = ORIGINAL_EVENTO.Hora_cierre_1[i]
    
     # Get open time for the first resource
    ha2 = ORIGINAL_EVENTO.Hora_apertura_2[i]
                        
    # Get close time for the first resource
    hc2 = ORIGINAL_EVENTO.Hora_cierre_2[i]
                        
    diff_hour1 = (hc1 - ha1)
    diff_hour1 = diff_hour1 / np.timedelta64(1, 'm')
    
    diff_hour2 = (hc2 - ha2)
    diff_hour2 = diff_hour2 / np.timedelta64(1, 'm')
    
    if diff_hour1 >= 360 or diff_hour2 >= 360:
        drop_list.append(i)
        
# Get the two datasets (first with short durations and second with large duration)
EVENTO = ORIGINAL_EVENTO[~ORIGINAL_EVENTO['Código'].isin(drop_list)]
EVENTO_HOURLESS = ORIGINAL_EVENTO[ORIGINAL_EVENTO['Código'].isin(drop_list)]

In [8]:
# Size of the individual
SIZE_INDIVIDUAL = EVENTO.shape[0]

# Number of generations
NUM_GENERATIONS = 20

# Optimal value to obtain (not reached)
MAX_VALUE = SIZE_INDIVIDUAL + 10 * SIZE_INDIVIDUAL

# Poblation members
SIZE_POBLATION = 1000

# CXPB  is the probability with which two individuals are crossed
CXPB = 0.5

# MUTPB is the probability for mutating an individual
MUTPB = 0.2

In [9]:
# Define the fitness function for maximizing the result
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creation of the individual with the fitness function
creator.create("Individual", list, fitness=creator.FitnessMax)

In [10]:
# Create a toolbox for the genetic algorithm
toolbox = base.Toolbox()

# Attribute generator having only 0, 1 or N days of the current event in the body of the individual (no day, first day or N day)
toolbox.register("attr_int", random.randint, 0, DAYS_EVENT)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, SIZE_INDIVIDUAL)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [11]:
# Enable multiprocessing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [12]:
# Function to find overlapped hours between resources
def find_hour_overlapping(ha1, hc1, ha2, hc2):
    found = False
    if (ha1 == ha2) or (ha2 < ha1 < hc2) or (ha1 < ha2 < hc1) or (ha2 < ha1 < hc1 < hc2) \
        or (ha1 < ha2 < hc2 < hc1):
        found = True
    return found

In [13]:
# Function to find overlapped hours in the first day
def find_first_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_1[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_1[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_1[EVENTO.index[j]]

     # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_1[EVENTO.index[j]]
                        
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [14]:
def find_second_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_2[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_2[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_2[EVENTO.index[j]]

    # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_2[EVENTO.index[j]]
        
    # Exclude resources that are not active in the second 
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [15]:
def find_mixed_time_overlapping(i, j):
    found = False
    
    # Get open time for the first resource
    ha1 = EVENTO.Hora_apertura_1[EVENTO.index[i]]

    # Get close time for the first resource
    hc1 = EVENTO.Hora_cierre_1[EVENTO.index[i]]

    # Get open time for the second resource
    ha2 = EVENTO.Hora_apertura_2[EVENTO.index[j]]

    # Get close time for the second resource
    hc2 = EVENTO.Hora_cierre_2[EVENTO.index[j]]
    
    # Exclude resources that are not active in the second 
    if int(ha1.hour) == 0 and int(hc1.hour) == 0:
        found = True
    else:    
        found = find_hour_overlapping(ha1, hc1, ha2, hc2)
    return found

In [16]:
# Function to find overlapped hours in the second day
def find_time_overlapping(i, j):
    found = False
    if find_first_time_overlapping(i, j) or find_second_time_overlapping(i, j) or find_mixed_time_overlapping(i, j):
        found = True
    return found

In [17]:
# Overlapping function to detect time problems in resources
def find_overlapping(individual):
    found = False
    day = 1
    while day <= DAYS_EVENT and not found:
        i = 0
        while i < len(individual) and not found:
            # If the resource is selected for the algorithm for day N
            if individual[i] == day:
                j = 0
                while j < len(individual) and not found:
                    if i != j:
                        if individual[j] == day:
                            found = find_time_overlapping(i, j)
                    j = j + 1
            i = i + 1
        day = day + 1
    return found

In [18]:
# Fitness function
def evalOneMax(individual):
    summa = 0
    if not find_overlapping(individual):
        for i in range(0, len(individual)):
            #ha = EVENTO.Hora_apertura_2[EVENTO.index[i]]
            #hc = EVENTO.Hora_cierre_2[EVENTO.index[i]]
            # Only if the resource is active in the second day it is considered
            #if ha.hour != 0 and hc != 0:
            #    summa = 1 + summa + PREFERENCIAS[PREFERENCIAS['Id_Recurso'] == EVENTO.index[i]].Preferencia.values[0]
            #else:
            if individual[i] != 0:
                summa = 1 + summa + PREFERENCIAS[PREFERENCIAS['Id_Recurso'] == EVENTO.index[i]].Preferencia.values[0]
    return summa,

In [20]:
# Record the allowed operations
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [21]:
# Initial population
pop = toolbox.population(n=SIZE_POBLATION)
for i in range(0, SIZE_POBLATION):
    pop[i] =  creator.Individual(SIZE_INDIVIDUAL * [0])

In [22]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))

In [23]:
# Store the values of the fitness function for each individual
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [24]:
# Function for calculating the stats each generation
def calculate_statistics(fits, g=0):
    print("-- Generation %i --" % g)   
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

In [25]:
# Gather all the fitnesses in one list and print the stats
fits = [ind.fitness.values[0] for ind in pop]
calculate_statistics(fits)

-- Generation 0 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0


In [26]:
# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < MAX_VALUE and g < NUM_GENERATIONS:
    # A new generation
    g = g + 1
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    calculate_statistics(fits, g)

-- Generation 1 --
  Min 0.0
  Max 5.301590165446928
  Avg 0.2109370618644254
  Std 0.700492176834279
-- Generation 2 --
  Min 0.0
  Max 5.760694768169368
  Avg 0.6385338677283229
  Std 1.121899745491352
-- Generation 3 --
  Min 0.0
  Max 6.536894077734139
  Avg 1.2249706962975975
  Std 1.3613090120458418
-- Generation 4 --
  Min 0.0
  Max 7.02250906720672
  Avg 1.8721370420769516
  Std 1.6042239870851542
-- Generation 5 --
  Min 0.0
  Max 8.371161453647334
  Avg 2.3388258119535927
  Std 1.8669052115590443
-- Generation 6 --
  Min 0.0
  Max 8.371161453647334
  Avg 2.829520278300789
  Std 2.170376201340096
-- Generation 7 --
  Min 0.0
  Max 11.367637564045463
  Avg 3.438143418296055
  Std 2.4506940572613725
-- Generation 8 --
  Min 0.0
  Max 11.367637564045463
  Avg 3.8354152385035802
  Std 2.757535100028005
-- Generation 9 --
  Min 0.0
  Max 11.367637564045463
  Avg 4.38107396703293
  Std 3.038191803506421
-- Generation 10 --
  Min 0.0
  Max 11.367637564045463
  Avg 4.795510148745572
 

In [27]:
# Find and print best individual
best_index = fits.index(max(fits))
best_partial_individual = pop[best_index]
print(best_partial_individual)

[0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [28]:
# Print the selected resources by the genetic algorithm for the two days
for i in range(0,len(best_partial_individual)):
    if best_partial_individual[i] != 0:
        print(EVENTO.Código[EVENTO.index[i]], best_partial_individual[i], EVENTO.Hora_apertura_1[EVENTO.index[i]], EVENTO.Hora_cierre_1[EVENTO.index[i]], EVENTO.Hora_apertura_2[EVENTO.index[i]], EVENTO.Hora_cierre_2[EVENTO.index[i]])



2 2 1900-01-01 10:00:00 1900-01-01 11:00:00 1900-01-01 10:00:00 1900-01-01 11:00:00
3 2 1900-01-01 11:15:00 1900-01-01 11:45:00 1900-01-01 11:15:00 1900-01-01 11:45:00
13 1 1900-01-01 10:00:00 1900-01-01 10:45:00 1900-01-01 10:00:00 1900-01-01 10:45:00
25 2 1900-01-01 12:00:00 1900-01-01 12:30:00 1900-01-01 12:00:00 1900-01-01 12:30:00
26 1 1900-01-01 12:45:00 1900-01-01 13:15:00 1900-01-01 12:45:00 1900-01-01 13:15:00
27 2 1900-01-01 13:30:00 1900-01-01 13:45:00 1900-01-01 13:30:00 1900-01-01 13:45:00
28 1 1900-01-01 14:30:00 1900-01-01 14:45:00 1900-01-01 14:30:00 1900-01-01 14:45:00
41 1 1900-01-01 13:30:00 1900-01-01 14:00:00 1900-01-01 13:30:00 1900-01-01 14:00:00
43 2 1900-01-01 17:30:00 1900-01-01 19:00:00 1900-01-01 17:30:00 1900-01-01 19:00:00
52 2 1900-01-01 19:30:00 1900-01-01 21:00:00 1900-01-01 19:30:00 1900-01-01 21:00:00
58 1 1900-01-01 19:00:00 1900-01-01 23:00:00 1900-01-01 19:00:00 1900-01-01 23:00:00
61 1 1900-01-01 16:00:00 1900-01-01 17:30:00 1900-01-01 16:00:00 19

In [29]:
# Add the large duration resources
best_individual = copy.copy(best_partial_individual)
for value in list(EVENTO_HOURLESS.Código):
    best_individual.insert(value,'#')

In [30]:
# Print the selected resources including short and long duration ones
for i in range(0,len(best_individual)):
    if best_individual[i] == '#' or best_individual[i] == 1:
        print(ORIGINAL_EVENTO.Código[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Fecha_1[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Hora_apertura_1[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Hora_cierre_1[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Nombre[ORIGINAL_EVENTO.index[i]])
    
for i in range(0,len(best_individual)):
    if best_individual[i] == 2:
        print(ORIGINAL_EVENTO.Código[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Fecha_2[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Hora_apertura_2[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Hora_cierre_2[ORIGINAL_EVENTO.index[i]], ORIGINAL_EVENTO.Nombre[ORIGINAL_EVENTO.index[i]])

5 2022-12-16 00:00:00 1900-01-01 15:00:00 1900-01-01 21:00:00 Road to Gamergy: Smash Bros Ultimate Open
12 2022-12-16 00:00:00 1900-01-01 00:00:00 1900-01-01 00:00:00 Open Pokémon Escarlata y Púrpura
13 2022-12-16 00:00:00 1900-01-01 10:00:00 1900-01-01 10:45:00 Inscripciones Pokémon TGC
24 2022-12-16 00:00:00 1900-01-01 12:00:00 1900-01-01 20:00:00 National League CSGO
26 2022-12-16 00:00:00 1900-01-01 12:45:00 1900-01-01 13:15:00 Valorant (Fiebre de la Spike) 1v1 Dieciseisavos de Final Bo1
28 2022-12-16 00:00:00 1900-01-01 14:30:00 1900-01-01 14:45:00 Valorant (Fiebre de la Spike) 1v1 Cuartos de Final Bo1
41 2022-12-16 00:00:00 1900-01-01 13:30:00 1900-01-01 14:00:00 Lucha de Sables
58 2022-12-16 00:00:00 1900-01-01 19:00:00 1900-01-01 23:00:00 Viewing Party Koi Vs Fnatic
61 2022-12-16 00:00:00 1900-01-01 16:00:00 1900-01-01 17:30:00 Gamergy Stars: Valorant
69 2022-12-16 00:00:00 1900-01-01 15:30:00 1900-01-01 16:00:00 Presentación C1b3r Wall Policia Nacional
73 2022-12-16 00:00:00 1